# ALL_RSS

### Given that RSS FEEDs update frequently we need to find a way to include the new jobs into the current DB without losing the "old" jobs.

#### Key DataFrames:

######  df = current output of crawler

###### df_a = the previous output of crawler

##### df1_copy = df + df_a (saved to be used by df_a in next iteration)

##### df1 = df + df_a  -> final df 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import mysql.connector
import psycopg2

In [2]:
df = pd.read_csv('/Users/juanreyesgarcia/Library/CloudStorage/OneDrive-FundacionUniversidaddelasAmericasPuebla/DEVELOPER/PROJECTS/CRAWLER_ALL/yummy_soup_rss.csv')

In [3]:
df_a = pd.read_csv('/Users/juanreyesgarcia/Library/CloudStorage/OneDrive-FundacionUniversidaddelasAmericasPuebla/DEVELOPER/PROJECTS/CRAWLER_ALL/RSS_CRAWLERS/yummy_soup_rss_copy.csv')

##### Play with the settings...

In [4]:
pd.set_option('display.max_colwidth', 150)

In [5]:
pd.set_option("display.max_rows", None)

# EDA Checklist 

In [6]:
df.head()

,title,link,description,pubDate,location
0,Senior Clojure Developer Codete,https://ClojureJobboard.com/clojure-job/ib-remote-senior-clojure-developer-krakow-codete-remotework.html,Remote Codete is not just a software company its a place where techenthusiasts can grow by doing what they love and feel valued for what they are...,NaN,NaN
1,Senior Software Engineer CIS Cloud Services Centripetal,https://ClojureJobboard.com/clojure-job/ha-remote-senior-software-engineer-cis-cloud-services-centripetal-remotework.html,Remote Do you want to be at the forefront of intelligencedriven cybersecurity We at Centripetal are innovators of disruptive cybersecurity soluti...,NaN,NaN
2,FullStack Clojure Developer Appear,https://ClojureJobboard.com/clojure-job/ia-remote-europe-full-stack-clojure-developer-appear-remotework.html,Remote At Appear were looking for a FullStack Clojure Developer to join our team Fulltime Oslo OR Remote About the role Were looking for a Cloju...,NaN,NaN
3,Software Engineer Clojure Backend Comcast,https://ClojureJobboard.com/clojure-job/ic-software-engineer-clojure-backend-philadelphia-comcast.html,Philadelphia United States Comcasts Technology Product Experience organization works at the intersection of media and technology Our innovative ...,NaN,NaN
4,Senior Application Developer Clojure Pennymac,https://ClojureJobboard.com/clojure-job/ii-senior-application-developer-clojure-agoura-hills-pennymac.html,Agoura Hills United States At Pennymac our people are the foundation of our success and at the heart of our dynamic work culture Together we work...,NaN,NaN


In [7]:
df.tail()

,title,link,description,pubDate,location
401,Director Global Channel Sales,https://cryptojobslist.com/jobs/director-global-channel-sales-chainalysis-new-york-city-new-york-remote,Tags Cryptocurrency Jobs Blockchain Remote Jobs Web3 Sales Jobs Web3 Community Jobs Cryptocurrency Director Jobs Blockchain Data Jobs Crypt...,Fri 03 Feb 2023 021131 GMT,New York City New York Remote
402,Test Automation Engineer,https://cryptojobslist.com/jobs/test-automation-engineer-coinshift-remote,Tags Cryptocurrency Jobs Web3 Remote Jobs Blockchain Developer Jobs Web3 Engineering Jobs Blockchain Web3 Jobs Web3 Frontend Jobs Blockchai...,Fri 03 Feb 2023 021130 GMT,Remote
403,Marketing Lead APAC,https://cryptojobslist.com/jobs/marketing-lead-apac-aptos-remote-asia,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Cryptocurrency Marketing Jobs Cryptocurrency Research Jobs Web3 Operations Jobs Web3 Web...,Mon 03 Oct 2022 030350 GMT,Remote Asia
404,Marketing Manager,https://cryptojobslist.com/jobs/marketing-manager-aptos-palo-alto-or-new-york,Tags Cryptocurrency Jobs Cryptocurrency Marketing Jobs Cryptocurrency Content Jobs Cryptocurrency Engineering Jobs Cryptocurrency Web3 Jobs ...,Fri 03 Feb 2023 021127 GMT,Palo Alto or New York
405,LATAM Ecosystem,https://cryptojobslist.com/jobs/latam-ecosystem-aptos-latin-america,Tags Cryptocurrency Jobs Web3 Community Jobs Web3 Developer Jobs Cryptocurrency Full Time Jobs About The Role Latin America is a quickly emerg...,Fri 03 Feb 2023 021126 GMT,Latin America


### Describe the DF (current)

In [8]:
df.describe()

,title,link,description,pubDate,location
count,406,406,404,391,308
unique,390,406,397,381,127
top,Executive Assistant,https://ClojureJobboard.com/clojure-job/ib-remote-senior-clojure-developer-krakow-codete-remotework.html,Seattle WA,Sat 04 Feb 2023 021103 GMT,Remote
freq,4,1,3,2,40


### Describe DF_a (previous)

In [9]:
df_a.describe()

,title,link,description,pubDate,location
count,1142,1142,1006,662,414
unique,1049,1142,968,33,119
top,Senior Software Engineer,https://cryptojobslist.com/jobs/network-engineer-binance-singapore,United States,2023-01-31,Worldwide
freq,9,1,6,67,101


### Checking for missing values DF (current)

In [10]:
df.isnull().sum()

title           0
link            0
description     2
pubDate        15
location       98
dtype: int64

### Checking for missing values DF_a (previous)

In [11]:
df_a.isnull().sum()

title            0
link             0
description    136
pubDate        480
location       728
dtype: int64

### Fill missing values with "NaN" DF (current)

In [12]:
# Fill missing values with "NaN"
df.fillna("NaN", inplace=True)

In [13]:
df.isnull().sum()

title          0
link           0
description    0
pubDate        0
location       0
dtype: int64

### Fill missing values with "NaN" DF_a (previous)

In [14]:
# Fill missing values with "NaN"
df_a.fillna("NaN", inplace=True)

In [15]:
df_a.isnull().sum()

title          0
link           0
description    0
pubDate        0
location       0
dtype: int64

### Checking for duplicates DF (current)

In [16]:
df.duplicated().sum()

0

### Checking for duplicates DF_a (previous)

In [17]:
df_a.duplicated().sum()

0

# From string to date time...

In [18]:
df_a.loc[:, 'pubDate']  = pd.to_datetime(df_a['pubDate'], errors="coerce", format="%a %d %b %Y", exact=False)
df.loc[:, 'pubDate']  = pd.to_datetime(df['pubDate'], errors="coerce", format="%a %d %b %Y", exact=False)

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_92264/1459423331.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_a.loc[:, 'pubDate']  = pd.to_datetime(df_a['pubDate'], errors="coerce", format="%a %d %b %Y", exact=False)
/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_92264/1459423331.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'pubDate']  = pd.to_datetime(df['pubDate'], errors="coerce", format="%a %d %b %Y", exact=False)


## Filter rows by a date range

In [19]:
# filter rows by a date range
start_date = pd.to_datetime('2016-01-01')
end_date = pd.to_datetime('2023-01-24')

#for df_a
date_range_filter_a = (df_a['pubDate'] >= start_date) & (df_a['pubDate'] <= end_date)
df_a = df_a.loc[~date_range_filter_a]

#for df
date_range_filter = (df['pubDate'] >= start_date) & (df['pubDate'] <= end_date)
df = df.loc[~date_range_filter]

In [20]:
df_a = df_a.sort_values(by='pubDate')
df = df.sort_values(by='pubDate')

In [21]:
df.describe(datetime_is_numeric=True)

,pubDate
count,267
mean,2023-02-16 08:43:08.764045056
min,2023-01-27 00:00:00
25%,2023-02-09 00:00:00
50%,2023-02-17 00:00:00
75%,2023-02-23 00:00:00
max,2023-03-02 00:00:00


In [22]:
df_a.describe(datetime_is_numeric=True)

,pubDate
count,0
mean,NaT
min,NaT
25%,NaT
50%,NaT
75%,NaT
max,NaT


In [23]:
df

,title,link,description,pubDate,location
130,DeFi Researcher Data Scientist,https://cryptojobslist.com/jobs/defi-researcher-data-scientist-almanak-remote,Tags Cryptocurrency Jobs Web3 Remote Jobs Blockchain Research Jobs Blockchain Data Jobs Web3 Defi Jobs At Almanak we utilize knowledge data a...,2023-01-27,NaN
78,Senior Associate Strategic Initiatives at CodePath Anywhere,https://www.crunchboard.com/jobs/191673575-senior-associate-strategic-initiatives-at-codepath,Reporting to CEO Location Remote Application close date Open Until Filled CodePath is a national nonprofit that brings together employers,2023-02-01,NaN
77,IT Support Technician at Alcoholics Anonymous World Services Inc New York NY USA,https://www.crunchboard.com/jobs/192004360-it-support-technician-at-alcoholics-anonymous-world-services-inc,About Alcoholics Anonymous Founded in 1935 Alcoholics Anonymous World Services AAWS is a 501c3 nonprofit organization and an international,2023-02-02,NaN
405,LATAM Ecosystem,https://cryptojobslist.com/jobs/latam-ecosystem-aptos-latin-america,Tags Cryptocurrency Jobs Web3 Community Jobs Web3 Developer Jobs Cryptocurrency Full Time Jobs About The Role Latin America is a quickly emerg...,2023-02-03,Latin America
76,POLICY RESEARCH ANALYST EDUCATIONAL EQUITY at Catalyst California Los Angeles CA USA,https://www.crunchboard.com/jobs/192302088-policy-research-analyst-educational-equity-at-catalyst-california,This is currently a workfromhome position however Catalyst California will shift to a hybrid workplace model once it is safe to do so you are,2023-02-03,NaN
404,Marketing Manager,https://cryptojobslist.com/jobs/marketing-manager-aptos-palo-alto-or-new-york,Tags Cryptocurrency Jobs Cryptocurrency Marketing Jobs Cryptocurrency Content Jobs Cryptocurrency Engineering Jobs Cryptocurrency Web3 Jobs ...,2023-02-03,Palo Alto or New York
399,Project Communications Lead,https://cryptojobslist.com/jobs/project-communications-lead-efforthr-remote,Tags Cryptocurrency Jobs Cryptocurrency Marketing Jobs Web3 Community Jobs Cryptocurrency Community Manager Jobs A fabulous new opportunity ha...,2023-02-03,NaN
401,Director Global Channel Sales,https://cryptojobslist.com/jobs/director-global-channel-sales-chainalysis-new-york-city-new-york-remote,Tags Cryptocurrency Jobs Blockchain Remote Jobs Web3 Sales Jobs Web3 Community Jobs Cryptocurrency Director Jobs Blockchain Data Jobs Crypt...,2023-02-03,New York City New York Remote
402,Test Automation Engineer,https://cryptojobslist.com/jobs/test-automation-engineer-coinshift-remote,Tags Cryptocurrency Jobs Web3 Remote Jobs Blockchain Developer Jobs Web3 Engineering Jobs Blockchain Web3 Jobs Web3 Frontend Jobs Blockchai...,2023-02-03,Remote
398,IT Security Officer Remote,https://cryptojobslist.com/jobs/it-security-officer-remote-keyrock-brussels,Tags Cryptocurrency Jobs Web3 Security Jobs Blockchain Full Time Jobs Blockchain Trading Jobs We are looking for an experienced IT security pr...,2023-02-03,Brussels


In [24]:
df.dtypes

title                  object
link                   object
description            object
pubDate        datetime64[ns]
location               object
dtype: object

In [25]:
df['title'] = df['title'].astype(str)
df['link'] = df['link'].astype(str)
df['description'] = df['description'].astype(str)
df['location'] = df['location'].astype(str)

In [26]:
df.dtypes

title                  object
link                   object
description            object
pubDate        datetime64[ns]
location               object
dtype: object

###### Remember 'object' is the pandas data type that is used to represent string data.

In [27]:
df1 = df.copy()

In [28]:
df['description'] = df['description'].str.slice(0, 1000)

# PostgreSQL

### This code creates a new table per iteration

In [29]:
# create a connection to the PostgreSQL database
cnx = psycopg2.connect(user='postgres', password='3312', host='localhost', port='5432', database='postgres')

# create a cursor object
cursor = cnx.cursor()

# get the name of the next table to create
get_table_name_query = '''
    SELECT COUNT(*) FROM information_schema.tables
    WHERE table_name LIKE 'mytable_%'
'''
cursor.execute(get_table_name_query)
result = cursor.fetchone()
next_table_number = result[0] + 1
next_table_name = 'mytable_{}'.format(next_table_number)

# prepare the SQL query to create a new table
create_table_query = '''
    CREATE TABLE {} (
        title VARCHAR(255),
        link VARCHAR(255),
        description VARCHAR(1000),
        pubDate TIMESTAMP,
        location VARCHAR(255)
    )
'''.format(next_table_name)

# execute the create table query
cursor.execute(create_table_query)

# insert the DataFrame into the PostgreSQL database as a new table
for index, row in df.iterrows():
    # check if pubDate column contains 'NaT'
    if pd.isna(row['pubDate']):
        continue  # skip this row
    else:
        insert_query = '''
            INSERT INTO {} (title, link, description, pubDate, location)
            VALUES (%s, %s, %s, %s, %s)
        '''.format(next_table_name)
        values = (row['title'], row['link'], row['description'], row['pubDate'], row['location'])
        cursor.execute(insert_query, values)

# commit the changes to the database
cnx.commit()

# close the cursor and connection
cursor.close()
cnx.close()



# MERGE 

In [30]:
#merged = pd.merge(df, df_a, on=['title','link','description','pubDate','location'], how='outer')

In [31]:
#merged.describe()

#### Check for duplicatesss...

In [32]:
#merged.duplicated(subset='link').sum()

#### Drop them...

In [33]:
#df1 = merged.drop_duplicates(subset=['link'])

In [34]:
#df1.duplicated(subset='link').sum()

In [35]:
#df1.describe()

# Save a copy of the df back into yummy_soup_rss_copy.csv so in the next iterations you will merge the output of the crawler with this df (basically making a loop)

## By doing this we will be able to merge the df that comes from the crawler and the merged df. If this was not done (& we just call the yummy_soup_rss_copy.csv file we would only merge those rows with the new ones, without including the data that was already merged)

In [36]:
#df1_copy = df1.copy()
#df1_copy.to_csv('yummy_soup_rss_copy.csv', index=False)

### Explore categorical data

For categorical data, check the number of unique values in each column 

In [37]:
#df1.nunique()

In [38]:
# Visualise the distribution of a categorical column using a bar chart
#df1['pubDate'].value_counts().plot(kind='bar', figsize=(14, 8))
#plt.show()

### Sort the dataframe by the date column in ascending order

In [39]:
#df1 = df1.sort_values(by='pubDate', ascending=False)

### Reset index...

In [40]:
#df1.reset_index(drop=True)

# GIVE ME THOSE JOBSSSS

## Only WorldWide - 2023

#### Given that some dates are NaN it is impossible to create a perfectly tailored df of solely worldwide jobs

##### I have tried to narrow down the search but there is still some jobs that are not fully worldwide. Although, the majority is indeed from anywhere in the world.

In [41]:
key_words = ['worldwide', 'international', 'anywhere']
regex = '|'.join(key_words)


worldwide = df1[df1['title'].str.contains(regex, case=False) | df1['description'].str.contains(regex, case=False) | df1['location'].str.contains(regex, case=False)]

In [42]:
worldwide.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_92264/3825536958.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  worldwide.describe()


,title,link,description,pubDate,location
count,49,49,49,49,49
unique,49,49,49,21,27
top,DeFi Researcher Data Scientist,https://cryptojobslist.com/jobs/defi-researcher-data-scientist-almanak-remote,Tags Cryptocurrency Jobs Web3 Remote Jobs Blockchain Research Jobs Blockchain Data Jobs Web3 Defi Jobs At Almanak we utilize knowledge data a...,2023-03-02 00:00:00,NaN
freq,1,1,1,7,12
first,NaN,NaN,NaN,2023-01-27 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-02 00:00:00,NaN


In [43]:
worldwide

,title,link,description,pubDate,location
130,DeFi Researcher Data Scientist,https://cryptojobslist.com/jobs/defi-researcher-data-scientist-almanak-remote,Tags Cryptocurrency Jobs Web3 Remote Jobs Blockchain Research Jobs Blockchain Data Jobs Web3 Defi Jobs At Almanak we utilize knowledge data a...,2023-01-27,NaN
78,Senior Associate Strategic Initiatives at CodePath Anywhere,https://www.crunchboard.com/jobs/191673575-senior-associate-strategic-initiatives-at-codepath,Reporting to CEO Location Remote Application close date Open Until Filled CodePath is a national nonprofit that brings together employers,2023-02-01,NaN
77,IT Support Technician at Alcoholics Anonymous World Services Inc New York NY USA,https://www.crunchboard.com/jobs/192004360-it-support-technician-at-alcoholics-anonymous-world-services-inc,About Alcoholics Anonymous Founded in 1935 Alcoholics Anonymous World Services AAWS is a 501c3 nonprofit organization and an international,2023-02-02,NaN
378,Customer Marketing Manager,https://cryptojobslist.com/jobs/customer-marketing-manager-chainalysis-remote-usa,Tags Cryptocurrency Jobs Blockchain Remote Jobs Cryptocurrency Marketing Jobs Cryptocurrency Sales Jobs Web3 Content Jobs Blockchain Operati...,2023-02-04,Remote USA
388,Associate General Counsel ProductRegulatory,https://cryptojobslist.com/jobs/associate-general-counsel-product-regulatory-aptos-palo-alto-new-york,Tags Cryptocurrency Jobs Blockchain Non Tech Jobs Blockchain Marketing Jobs Blockchain Research Jobs Web3 Legal Jobs Cryptocurrency Engineer...,2023-02-04,Palo Alto New York
364,General Interest,https://cryptojobslist.com/jobs/general-interest-chainsafe-systems-remote,Tags Cryptocurrency Jobs Web3 Remote Jobs Cryptocurrency Research Jobs Blockchain Community Jobs Cryptocurrency Developer Jobs Web3 Web3 Job...,2023-02-05,Remote
357,Cybercrimes Investigator,https://cryptojobslist.com/jobs/cybercrimes-investigator-chainalysis-washington-d-c-remote,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Cryptocurrency Research Jobs Cryptocurrency Sales Jobs Cryptocurrency Engineering Jobs W...,2023-02-06,Washington DC Remote
75,Director of Health Plan Strategy Sales New 911 Telehealth Technology at MD Ally Technologies Inc Anywhere,https://www.crunchboard.com/jobs/193311778-director-of-health-plan-strategy-sales-new-911-telehealth-technology-at-md-ally-technologies-inc,ABOUT MD ALLY Each year millions of nonemergency calls are made to 911 that decrease ambulance availability overcrowd EDs and drive billions in,2023-02-06,NaN
351,Community Communication Manager,https://cryptojobslist.com/jobs/community-communication-manager-request-paris-france,Tags Cryptocurrency Jobs Cryptocurrency Community Jobs Cryptocurrency Community Manager Jobs Web3 Content Jobs Web3 Operations Jobs Web3 Bus...,2023-02-07,Paris France
339,Targeted ads manager expert with Crowdfunding experience,https://cryptojobslist.com/jobs/targeted-ads-manager-expert-with-crowdfunding-experience-gd10-capital-worldwide,Tags Cryptocurrency Jobs Web3 Marketing Jobs Blockchain Content Jobs Cryptocurrency Social Media Jobs The Role We are looking for an exception...,2023-02-08,Worldwide


## Analyst - WorldWide

In [44]:

analyst = worldwide[worldwide['title'].str.contains('analyst', case=False) | worldwide['description'].str.contains('analyst', case=False)]


In [45]:
analyst.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_92264/1449045089.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  analyst.describe()


,title,link,description,pubDate,location
count,3,3,3,3,3
unique,3,3,3,3,3
top,Cybercrimes Investigator,https://cryptojobslist.com/jobs/cybercrimes-investigator-chainalysis-washington-d-c-remote,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Cryptocurrency Research Jobs Cryptocurrency Sales Jobs Cryptocurrency Engineering Jobs W...,2023-02-06 00:00:00,Washington DC Remote
freq,1,1,1,1,1
first,NaN,NaN,NaN,2023-02-06 00:00:00,NaN
last,NaN,NaN,NaN,2023-02-23 00:00:00,NaN


In [46]:
analyst

,title,link,description,pubDate,location
357,Cybercrimes Investigator,https://cryptojobslist.com/jobs/cybercrimes-investigator-chainalysis-washington-d-c-remote,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Cryptocurrency Research Jobs Cryptocurrency Sales Jobs Cryptocurrency Engineering Jobs W...,2023-02-06,Washington DC Remote
241,Accounts Payable Analyst,https://cryptojobslist.com/jobs/accounts-payable-analyst-parity-technologies-london-england-united-kingdom,Tags Cryptocurrency Jobs Web3 Analyst Jobs Cryptocurrency Operations Jobs Cryptocurrency Finance Jobs Web3 Accounting Jobs Web3 Full Time Jo...,2023-02-17,London England United Kingdom
145,Binance Accelerator Programme Data analyst intern,https://cryptojobslist.com/jobs/binance-accelerator-programme-data-analyst-intern-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Internship Jobs Blockchain Analyst Jobs Blockchain Engineering Jobs Web3 Exc...,2023-02-23,Singapore


## Sales - Worldwide

In [47]:
sales = worldwide[worldwide['title'].str.contains('sales', case=False) | worldwide['description'].str.contains('sales', case=False)]

In [48]:
sales.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_92264/3195154911.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  sales.describe()


,title,link,description,pubDate,location
count,5,5,5,5,5
unique,5,5,5,4,5
top,Customer Marketing Manager,https://cryptojobslist.com/jobs/customer-marketing-manager-chainalysis-remote-usa,Tags Cryptocurrency Jobs Blockchain Remote Jobs Cryptocurrency Marketing Jobs Cryptocurrency Sales Jobs Web3 Content Jobs Blockchain Operati...,2023-02-06 00:00:00,Remote USA
freq,1,1,1,2,1
first,NaN,NaN,NaN,2023-02-04 00:00:00,NaN
last,NaN,NaN,NaN,2023-02-10 00:00:00,NaN


In [49]:
sales

,title,link,description,pubDate,location
378,Customer Marketing Manager,https://cryptojobslist.com/jobs/customer-marketing-manager-chainalysis-remote-usa,Tags Cryptocurrency Jobs Blockchain Remote Jobs Cryptocurrency Marketing Jobs Cryptocurrency Sales Jobs Web3 Content Jobs Blockchain Operati...,2023-02-04,Remote USA
357,Cybercrimes Investigator,https://cryptojobslist.com/jobs/cybercrimes-investigator-chainalysis-washington-d-c-remote,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Cryptocurrency Research Jobs Cryptocurrency Sales Jobs Cryptocurrency Engineering Jobs W...,2023-02-06,Washington DC Remote
75,Director of Health Plan Strategy Sales New 911 Telehealth Technology at MD Ally Technologies Inc Anywhere,https://www.crunchboard.com/jobs/193311778-director-of-health-plan-strategy-sales-new-911-telehealth-technology-at-md-ally-technologies-inc,ABOUT MD ALLY Each year millions of nonemergency calls are made to 911 that decrease ambulance availability overcrowd EDs and drive billions in,2023-02-06,NaN
340,Web designer developer frontend backend with Graphic design skills Writer,https://cryptojobslist.com/jobs/web-designer-developer-frontend-backed-with-graphic-design-skills-writer-gd10-capital-worldwide,Tags Cryptocurrency Jobs Web3 Marketing Jobs Blockchain Sales Jobs Web3 Content Jobs Web3 Developer Jobs Web3 Ui Ux Jobs Web3 Designer Jobs...,2023-02-08,Worldwide
313,Sales Representative,https://cryptojobslist.com/jobs/sales-representative-aurora-labs-remote-1,Tags Cryptocurrency Jobs Blockchain Remote Jobs Blockchain Sales Jobs Web3 Business Development Jobs Web3 Full Time Jobs Blockchain Evm Jobs...,2023-02-10,Remote


## Python - Worldwide

In [50]:
python = worldwide[worldwide['title'].str.contains('python', case=False) | worldwide['description'].str.contains('python', case=False)]

In [51]:
python.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_92264/349435123.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  python.describe()


,title,link,description,pubDate,location
count,3,3,3,3,3
unique,3,3,3,3,3
top,DeFi Researcher Data Scientist,https://cryptojobslist.com/jobs/defi-researcher-data-scientist-almanak-remote,Tags Cryptocurrency Jobs Web3 Remote Jobs Blockchain Research Jobs Blockchain Data Jobs Web3 Defi Jobs At Almanak we utilize knowledge data a...,2023-01-27 00:00:00,NaN
freq,1,1,1,1,1
first,NaN,NaN,NaN,2023-01-27 00:00:00,NaN
last,NaN,NaN,NaN,2023-02-23 00:00:00,NaN


In [52]:
python

,title,link,description,pubDate,location
130,DeFi Researcher Data Scientist,https://cryptojobslist.com/jobs/defi-researcher-data-scientist-almanak-remote,Tags Cryptocurrency Jobs Web3 Remote Jobs Blockchain Research Jobs Blockchain Data Jobs Web3 Defi Jobs At Almanak we utilize knowledge data a...,2023-01-27,NaN
240,Sr DevOps Engineer New Zealand Fiji GMT12,https://cryptojobslist.com/jobs/sr-devops-engineer-new-zealand-fiji-gmt-12-polygon-auckland,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Cryptocurrency Devops Jobs Blockchain Security Jobs Blockchain Developer Jobs Blockchain...,2023-02-17,Auckland
145,Binance Accelerator Programme Data analyst intern,https://cryptojobslist.com/jobs/binance-accelerator-programme-data-analyst-intern-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Internship Jobs Blockchain Analyst Jobs Blockchain Engineering Jobs Web3 Exc...,2023-02-23,Singapore


## SQL - Worldwide

In [53]:
sql = worldwide[worldwide['title'].str.contains('sql', case=False) | worldwide['description'].str.contains('sql', case=False)]

In [54]:
sql.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_92264/1629035654.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  sql.describe()


,title,link,description,pubDate,location
count,4,4,4,4,4
unique,4,4,4,4,4
top,Web designer developer frontend backend with Graphic design skills Writer,https://cryptojobslist.com/jobs/web-designer-developer-frontend-backed-with-graphic-design-skills-writer-gd10-capital-worldwide,Tags Cryptocurrency Jobs Web3 Marketing Jobs Blockchain Sales Jobs Web3 Content Jobs Web3 Developer Jobs Web3 Ui Ux Jobs Web3 Designer Jobs...,2023-02-08 00:00:00,Worldwide
freq,1,1,1,1,1
first,NaN,NaN,NaN,2023-02-08 00:00:00,NaN
last,NaN,NaN,NaN,2023-02-23 00:00:00,NaN


In [55]:
sql

,title,link,description,pubDate,location
340,Web designer developer frontend backend with Graphic design skills Writer,https://cryptojobslist.com/jobs/web-designer-developer-frontend-backed-with-graphic-design-skills-writer-gd10-capital-worldwide,Tags Cryptocurrency Jobs Web3 Marketing Jobs Blockchain Sales Jobs Web3 Content Jobs Web3 Developer Jobs Web3 Ui Ux Jobs Web3 Designer Jobs...,2023-02-08,Worldwide
301,Product Intern Pay Card Data,https://cryptojobslist.com/jobs/product-intern-pay-card-data-binance-asia,Tags Cryptocurrency Jobs Blockchain Remote Jobs Web3 Trading Jobs Web3 Internship Jobs Cryptocurrency Full Time Jobs Blockchain Part Time Jo...,2023-02-10,Asia
240,Sr DevOps Engineer New Zealand Fiji GMT12,https://cryptojobslist.com/jobs/sr-devops-engineer-new-zealand-fiji-gmt-12-polygon-auckland,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Cryptocurrency Devops Jobs Blockchain Security Jobs Blockchain Developer Jobs Blockchain...,2023-02-17,Auckland
145,Binance Accelerator Programme Data analyst intern,https://cryptojobslist.com/jobs/binance-accelerator-programme-data-analyst-intern-binance-singapore,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Internship Jobs Blockchain Analyst Jobs Blockchain Engineering Jobs Web3 Exc...,2023-02-23,Singapore
